<a href="https://colab.research.google.com/github/diogo-costa-silva/sql-murder-mystery/blob/main/SQL_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# getting the database from my github
!wget https://github.com/diogo-costa-silva/assets/raw/main/databases/sql-murder-mystery.db -O sql-murder-mystery.db

In [2]:
# Install ipython-sql
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.3 MB/s eta 0:00:00


In [ ]:
# Load the SQL extension
%load_ext sql

In [7]:
# Create a SQLite database
%sql sqlite:///sql-murder-mystery.db

In [4]:
# EXAMPLE -- Execute SQL commands to create a table, insert data, and run a query
%%sql
CREATE TABLE test_table(name, age);
INSERT INTO test_table VALUES('Alice', 24);
SELECT * FROM test_table;

 * sqlite://
Done.
1 rows affected.
Done.


name,age
Alice,24


### SQL Commands used in this exercise:

SELECT -- Extracts data from a database <br>
FROM -- Specify the location of the table you will extract the data <br>
INNER JOIN, ON -- keyword selects records that have matching values in both tables <br>
WHERE -- Clause used to filter the output <br>
ORDER BY -- keyword is used to sort the result in ascending or descending order <br>
LIKE -- Performs a partial string match <br>
INSERT INTO -- Adds new rows to a table <br>
VALUES -- Specifies the values to be inserted <br>
DISTINCT -- Filters out duplicate values from the result set <br>
GROUP BY -- Groups rows based on specified columns <br>
HAVING -- Filters groups based on aggregate functions <br>
COUNT -- Calculates the number of rows or non-null values <br>
AS -- Assigns an alias to a column or table <br>
AND -- Combines multiple conditions in a WHERE clause <br>
OR -- Combines multiple conditions, at least one of which must be true <br>

<br>

---

<br>

The initial clues provided are the following:

- The crime is murder
- The crime happened on Jan 15, 2018​
- Took place at SQL City

<br>

## Step 1

The first step is to check the “crime_scene_report” table where the city should be at **SQL City** at the time of **Jan 15, 2018**.

In [9]:
%%sql
SELECT type, description, city FROM crime_scene_report
WHERE date = 20180115 AND city = 'SQL City'

   sqlite://
 * sqlite:///sql-murder-mystery.db
Done.


type,description,city
assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!",SQL City
assault,Report Not Found,SQL City
murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


## Step 2

Since this is SQL Murder mystery, I should focus on the type of crime which is **murder**. A clue provided here in the “crime_scene_report” table is that there are **two witnesses**, the **first** one lives at the last house on **Northwestern Dr** and the **second**  who’s name is **Annabel** and lives somewhere on **Franklin Ave**.

Let’s find the **first witness** that lives at the **last house** on **Northwestern Dr**.



In [13]:
%%sql
SELECT name, address_number, address_street_name
FROM person
WHERE address_street_name LIKE '%Northwestern%'
ORDER BY address_number DESC
LIMIT 10;

   sqlite://
 * sqlite:///sql-murder-mystery.db
Done.


name,address_number,address_street_name
Morty Schapiro,4919,Northwestern Dr
Lasonya Wildey,3824,Northwestern Dr
Sophie Tiberio,3755,Northwestern Dr
Torie Thalmann,3697,Northwestern Dr
Coretta Cubie,3631,Northwestern Dr
Cody Schiel,3524,Northwestern Dr
Emmitt Aceuedo,3491,Northwestern Dr
Leonora Wolfsberger,3483,Northwestern Dr
Freddie Ellzey,3449,Northwestern Dr
Boris Bijou,3327,Northwestern Dr


## Step 3

Let’s check **Morty Schapiro’s** statement if we can get more leads to this crime.



In [14]:
%%sql
SELECT person.name, interview.transcript FROM interview
INNER JOIN person
ON interview.person_id = person.id
WHERE person.name = 'Morty Schapiro';

   sqlite://
 * sqlite:///sql-murder-mystery.db
Done.


name,transcript
Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


The clues provided by **Morty Schapiro** are the **gender**, **membership number** and **car plate**, but first let’s check the statement of <u>second witness</u>.

In [16]:
%%sql
SELECT name, address_number, address_street_name
FROM person
WHERE address_street_name LIKE '%Franklin Ave%' AND name LIKE 'Annabel%';

   sqlite://
 * sqlite:///sql-murder-mystery.db
Done.


name,address_number,address_street_name
Annabel Miller,103,Franklin Ave


## Step 3

We will now check **Annabel Miller’s** statement if we can get more leads to this crime.

In [17]:
%%sql
SELECT person.name, interview.transcript
FROM interview
INNER JOIN person
ON interview.person_id = person.id
WHERE person.name = 'Annabel Miller';

   sqlite://
 * sqlite:///sql-murder-mystery.db
Done.


name,transcript
Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


By having the statements of **both witnesses**, we can now query that the killer is a **man**, which has a **Gold** membership in the “Get Fit Now Gym” with a number that started with **"48Z"**. His car plate included **“H42W”**, and the second witness recognized him from when he was working out last week on January 9th (**20180109**).

Let's try to query this as a whole and use alias as this query get long.

In [19]:
%%sql
SELECT membership.name
FROM get_fit_now_member AS membership
INNER JOIN get_fit_now_check_in AS checkin ON checkin.membership_id = membership.id
INNER JOIN person ON membership.person_id = person.id
INNER JOIN drivers_license as drivlic ON person.license_id = drivlic.id
WHERE checkin.check_in_date = 20180109
 AND membership.membership_status = 'gold'
 AND drivlic.plate_number LIKE '%H42W%';

   sqlite://
 * sqlite:///sql-murder-mystery.db
Done.


name
Jeremy Bowers


In [ ]:
SELECT

## Step 4

By providing some of the conditions, we came up with a certain name of **Jeremy Bowers**. Lets try to connect the **gender** and the **membership number** and see if the query returs the same name or no data.

In [21]:
%%sql
SELECT membership.name, membership.id
FROM get_fit_now_member AS membership
INNER JOIN get_fit_now_check_in AS checkin ON checkin.membership_id = membership.id
INNER JOIN person ON membership.person_id = person.id
INNER JOIN drivers_license as drivlic ON person.license_id = drivlic.id
WHERE membership.name = "Jeremy Bowers"
 AND drivlic.gender = "male";

   sqlite://
 * sqlite:///sql-murder-mystery.db
Done.


name,id
Jeremy Bowers,48Z55


Given all of the conditions stated on the SQL query, we now found the suspect on the murder case. Let’s try to check if our answer is correct.

![Check-your-solution](https://raw.githubusercontent.com/diogo-costa-silva/sql-murder-mystery/main/images/check_your_solution_1.png)

## Step 5

I will now check Jeremy Bowers’ statement regarding to find the real villain behind this crime.

In [22]:
%%sql
SELECT person.name, interview.transcript
FROM interview
INNER JOIN person ON interview.person_id = person.id
WHERE person.name = 'Jeremy Bowers';

   sqlite://
 * sqlite:///sql-murder-mystery.db
Done.


name,transcript
Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


## Step 6

Given the statement of Mr. Jeremy Bowers, these are the clues that will lead to the perpetrator.
- Woman
- A lot of money (Annual Income)
- 5'5" — 5'7"
- Red hair
- Drives a Tesla Model S
- Attended SQL Symphony Concert 3 times, December 2017

I will try to create an SQL query that will relate to the statement using a single query.

In [24]:
%%sql
SELECT person.name, inc.annual_income,drivlic.height,
drivlic.car_make || ' ' || drivlic.car_model AS 'Car Make and Model',
  fbcheckin.event_name AS 'Event Name' FROM person
INNER JOIN drivers_license AS drivlic
ON person.license_id = drivlic.id
INNER JOIN income AS inc
ON person.ssn = inc.ssn
INNER JOIN facebook_event_checkin AS fbcheckin
ON fbcheckin.person_id = person.id
WHERE drivlic.hair_color = 'red'
 AND drivlic.gender = 'female'
 AND drivlic.car_make LIKE 'Tesla%'
 AND drivlic.car_model LIKE '%Model S%'
 AND drivlic.height BETWEEN 65 AND 67
 AND fbcheckin.date LIKE '201712%'
 AND fbcheckin.event_name = 'SQL Symphony Concert'
GROUP BY person.name
HAVING COUNT(fbcheckin.person_id) <= 3

   sqlite://
 * sqlite:///sql-murder-mystery.db
Done.


name,annual_income,height,Car Make and Model,Event Name
Miranda Priestly,310000,66,Tesla Model S,SQL Symphony Concert


Let’s check if our perpetrator is correct.

![Check-your-solution](https://raw.githubusercontent.com/diogo-costa-silva/sql-murder-mystery/main/images/check_your_solution_2.png)